In [1]:
import re
import os
import numpy as np
import pandas as pd
from pandas import DataFrame as df

au2kcal = 627.50957099203276

In [2]:
def W1_method(table,ZPE):
    
   #**************************
    SCF_DZ=table['ESCF'][0]
    SCF_TZ=table['ESCF'][1]
    SCF_QZ=table['ESCF'][2]
    CCSD_DZ=table['ECCSD'][0]
    CCSD_TZ=table['ECCSD'][1]
    CCSD_QZ=table['ECCSD'][2]
    CCSDT_DZ=table['ECCSDT'][0]
    CCSDT_TZ=table['ECCSDT'][1]
    FC_MTsmall=table['ECCSDT'][3]
    FULL_MTsmall=table['ECCSDT'][4]
    
  #*****************************
        #*****************************
    CCSD_CORR_DZ=CCSD_DZ-SCF_DZ
    CCSD_CORR_TZ=CCSD_TZ-SCF_TZ
    CCSD_CORR_QZ=CCSD_QZ-SCF_QZ
    
    CCSDT_CORR_DZ=CCSDT_DZ-CCSD_DZ
    CCSDT_CORR_TZ=CCSDT_TZ-CCSD_TZ
    #******************************
    SCF_temp = (SCF_QZ-SCF_TZ)/(((4/3)**5)-1)
    SCF_INF= SCF_QZ+SCF_temp
    
    CCSD_temp = (CCSD_CORR_QZ-CCSD_CORR_TZ)/(((4/3)**(3.22))-1)
    CCSD_INF = CCSD_CORR_QZ+CCSD_temp
    
    CCSDT_temp = (CCSDT_CORR_TZ-CCSDT_CORR_DZ)/(((3/2)**(3.22))-1)
    CCSDT_INF = CCSDT_CORR_TZ+CCSDT_temp
    
    CORE_CORR = FULL_MTsmall-FC_MTsmall
    
    ZPE_CORR = ZPE*0.985
    
    W1_electronic = SCF_INF+CCSD_INF+CCSDT_INF+CORE_CORR
    print(SCF_INF,CCSD_INF,CCSDT_INF,CORE_CORR)
    W1_0K = W1_electronic+ZPE_CORR
    #print("*************************************************")
    """
    print("Extrapolated_SCF is {}".format(SCF_INF))
    print("Extrapolated_CCSD is {}".format(CCSD_INF))
    print("Extrapolated_CCSDT is {}".format(CCSDT_INF))
    print("CORE Component is {}".format(CORE_CORR))
    print("W1 Electronic Energy is {}".format(W1_electronic))
     """
    #print("W1(0K) is {}".format(W1_0K))
    #print("***************************************************")
    return W1_0K

def make_df(data):
    values=data 
    column = ['ESCF',
             'ECCSD',
             'ECCSDT']
    row = ['VDZ','VTZ','VQZ','MTsmall','MTsmall_full']
    return pd.DataFrame(values,row,column)


## Molpro W1 Results for TS

- For a particular system, Molpro outupt files for TS are stored in '../SYSTEM_NAME/SYSTEM_NAME_TS_W1_FILES'.<br>  
   For example: '../ETHYL/ETHYL_TS_W1_FILES'<br>  

* All the steps are simlar to W1 energy calculation for reaction energies, described in Molpro_W1_Energy.ipynb. Only Files names are different.


<mark>Please go through ' Molpro_W1_Energy.ipynb ' notebook first.</mark>
 
  
  (1) After extraction of energies from molpro output file, they are stored in '../SYSTEM_NAME/SYSTEM_DATA/' dir
  
   File name is : Name of this csv file is: SYSTEM_NAME_TS_NAME_Molpro.csv .
   
      Example: Inside 'ETHYL/ETHYL_DATA' dir,   ETHYL_TS_01_molpro.csv, ETHYL_TS_02_molpro.csv, ETHYL_TS_03_molpro.csv, and ETHYL_TS_04_molpro.csv are stored. 
   
  (2) Name of Files after extrapolation steps:  SYSTEM_NAME_TS_W1.csv.
   
      For example: For ETHYL sytem, file name is : ETHYL_TS_W1.csv
   
   (3) Name of files containing,W1 energy of TS wrt R+O2:  SYSTEM_NAME_TS_W1_Rxn_E.csv.

       For example : For ETHYL system, file name is : ETHYL_TS_W1_Rxn_E.csv

In [3]:
DIR_OUTPUT = '../CSV_FILES/'


SYSTEMS = ['ETHYL','ISOPROPYL','ISOBUTYL','TERTBUTYL','NEOPENTYL','CYCLOHEXYL','CYCLOHEXENYL',
           'CYCLOHEXADIENYL']


MOLECULE = ['TS_01','TS_02','TS_03','TS_04']

METHOD = ['VDZ','VTZ','VQZ','MTsmall','MTsmall_full']
SYSTEMS_DF = []
for isystem in SYSTEMS:
    
    DIR_DATA = '../'+isystem+'/'+isystem+'_DATA/'
    ZPE_MOL = np.loadtxt(DIR_DATA+isystem+'_TS_ZPE.dat')

    DIR_OUT_FILES = '../'+isystem+'/'+isystem+'_TS_W1_FILES/'
    MOL_DF = []
    W1_DF = pd.DataFrame(np.zeros((len(MOLECULE),1)),MOLECULE,columns=['W1'])
    for imol in MOLECULE:
        INIT_DF = make_df(np.zeros((5,3)))
        for imethod in METHOD:
        
            FILENAME = DIR_OUT_FILES+imol+'_'+imethod+'.out'
            file = open(FILENAME,"r")
            #print(FILENAME)

            for line in file:
                if re.search('Reference energy',line):
                    #print(line)
                    INIT_DF['ESCF'][imethod]=float(line.split()[2])
                
                if re.search('RHF-UCCSD energy',line):
                    INIT_DF['ECCSD'][imethod] = float(line.split()[2])
                
                if re.search('!RHF-UCCSD\(T\) energy',line):
                    INIT_DF['ECCSDT'][imethod] = float(line.split()[2])
                
                if re.search("CCSD total energy",line):
                    INIT_DF['ECCSD'][imethod] = float(line.split()[3])
                
                if re.search("!CCSD\(T\) total energy",line):
                    INIT_DF['ECCSDT'][imethod] = float(line.split()[3])
        
        INIT_DF.index.name='BASIS'
        INIT_DF.to_csv(DIR_DATA+isystem+'_'+imol+'_TS_Molpro.csv')
        #INIT_DF.to_csv(DIR_DATA+imol+'_Molpro.csv')  
        W1_method(INIT_DF,ZPE_MOL[MOLECULE.index(imol)])
        W1_DF['W1'][imol] =  W1_method(INIT_DF,ZPE_MOL[MOLECULE.index(imol)])
        MOL_DF.append(INIT_DF)
    print(W1_DF)
    #print(DIR_OUTPUT+isystem+'_W1.csv')
    W1_DF.index.name='MOLECULE'
    W1_DF.to_csv(DIR_OUTPUT+isystem+'_TS_W1.csv')
    SYSTEMS_DF.append(MOL_DF)


/tmp/ipykernel_12238/3115014174.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SCF_DZ=table['ESCF'][0]
/tmp/ipykernel_12238/3115014174.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SCF_TZ=table['ESCF'][1]
/tmp/ipykernel_12238/3115014174.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SCF_QZ=table['ESCF'][2]
/tmp/ipykernel_12238/3115014174.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated

-228.24713543604315 -0.9441426176801982 -0.04385098624002145 -0.3437818065600027
-228.24713543604315 -0.9441426176801982 -0.04385098624002145 -0.3437818065600027
-228.2458512187765 -0.9487766080119389 -0.048770170310710854 -0.3440771093399917
-228.2458512187765 -0.9487766080119389 -0.048770170310710854 -0.3440771093399917
-228.25267450404002 -0.9453845943191256 -0.044908066927423634 -0.3439033621010026
-228.25267450404002 -0.9453845943191256 -0.044908066927423634 -0.3439033621010026
-228.2390073857695 -0.9594991460601547 -0.046824992720224744 -0.34372983992201966
-228.2390073857695 -0.9594991460601547 -0.046824992720224744 -0.34372983992201966
               W1
TS_01 -229.514765
TS_02 -229.523509
TS_03 -229.520514
TS_04 -229.523646
-267.3045217467614 -1.142984312351928 -0.05277349244728224 -0.4087316274619752
-267.3045217467614 -1.142984312351928 -0.05277349244728224 -0.4087316274619752
-267.3058573889469 -1.144992662218492 -0.057149754679125024 -0.40905216687599477
-267.3058573889469 

/tmp/ipykernel_12238/3115014174.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SCF_DZ=table['ESCF'][0]
/tmp/ipykernel_12238/3115014174.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SCF_TZ=table['ESCF'][1]
/tmp/ipykernel_12238/3115014174.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SCF_QZ=table['ESCF'][2]
/tmp/ipykernel_12238/3115014174.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated

In [4]:
SYSTEMS = ['ETHYL','ISOPROPYL','ISOBUTYL','TERTBUTYL','NEOPENTYL','CYCLOHEXYL','CYCLOHEXENYL',
           'CYCLOHEXADIENYL']
#SYSTEMS = ['ETHYL','ISOPROPYL','ISOBUTYL','TERTBUTYL','CYCLOHEXYL']
DATA_1 = pd.read_csv(DIR_OUTPUT+'O2_OH_OOH'+'_W1.csv',index_col='MOLECULE')

for isystem in SYSTEMS:
    DATA_2 = pd.read_csv(DIR_OUTPUT+isystem+'_TS_W1.csv',index_col='MOLECULE')
    DATA_3 = pd.read_csv(DIR_OUTPUT+isystem+'_W1.csv',index_col='MOLECULE')
    NAME_SPECIES = ['R+O2','TS_01','TS_02','TS_03','TS_04']
    
    W1_Rxn_DF = pd.DataFrame(np.zeros((len(NAME_SPECIES),1)),NAME_SPECIES,['W1'])
    E_ref = DATA_3['W1']['Mol_01']+DATA_1['W1']['O2']
    
    W1_Rxn_DF['W1']['TS_01'] = DATA_2['W1']['TS_01']-E_ref
    W1_Rxn_DF['W1']['TS_02'] = DATA_2['W1']['TS_02']-E_ref
    W1_Rxn_DF['W1']['TS_03'] = DATA_2['W1']['TS_03']-E_ref
    W1_Rxn_DF['W1']['TS_04'] = DATA_2['W1']['TS_04']-E_ref
    W1_Rxn_DF= W1_Rxn_DF*au2kcal
    W1_Rxn_DF.index.name='SPECIES'
    W1_Rxn_DF.to_csv(DIR_OUTPUT+isystem+'_TS_W1_Rxn_E.csv')

In [6]:
DATA = pd.read_csv(DIR_OUTPUT+'ETHYL'+'_TS_W1_Rxn_E.csv',index_col='SPECIES')

In [7]:
DATA

,W1
SPECIES,
R+O2,0.000000
TS_01,4.050993
TS_02,-1.436274
TS_03,0.443226
TS_04,-1.521828
